## Analysis of data with Spark

In [175]:
import findspark
findspark.init('/home/rich/spark/spark-2.4.3-bin-hadoop2.7')
import pandas as pd
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import matplotlib.pyplot as plt

### Data dictionary of flights data
    mon — month (integer between 1 and 12)
    dom — day of month (integer between 1 and 31)
    dow — day of week (integer; 1 = Monday and 7 = Sunday)
    org — origin airport (IATA code)
    mile — distance (miles)
    carrier — carrier (IATA code)
    depart — departure time (decimal hour)
    duration — expected duration (minutes)
    delay — delay (minutes)


### Loading flights data

In [176]:
file_path = './data/flights.csv'

In [177]:
#pandas is my first love :)
df = pd.read_csv(file_path)

In [178]:
df.head()

,mon,dom,dow,carrier,flight,org,mile,depart,duration,delay
0,11,20,6,US,19,JFK,2153,9.48,351,NaN
1,0,22,2,UA,1107,ORD,316,16.33,82,30.0
2,2,20,4,UA,226,SFO,337,6.17,82,-8.0
3,9,13,1,AA,419,ORD,1236,10.33,195,-5.0
4,4,2,5,AA,325,ORD,258,8.92,65,NaN


In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 10 columns):
mon         50000 non-null int64
dom         50000 non-null int64
dow         50000 non-null int64
carrier     50000 non-null object
flight      50000 non-null int64
org         50000 non-null object
mile        50000 non-null int64
depart      50000 non-null float64
duration    50000 non-null int64
delay       47022 non-null float64
dtypes: float64(2), int64(6), object(2)
memory usage: 3.8+ MB


In [180]:
# Create SparkSession object
spark = SparkSession.builder \
                    .master('local[*]') \
                    .appName('Flights') \
                    .getOrCreate()

In [181]:
# Read data from CSV file
flights = spark.read.csv(file_path,
                         sep=',',
                         header=True,
                         inferSchema=True,
                         nullValue='NA')

# Get number of records
print("The data contain %d records." % flights.count())

# View the first five records
flights.show(5)

# Check column data types
flights.dtypes

The data contain 50000 records.
+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351| null|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65| null|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows



[('mon', 'int'),
 ('dom', 'int'),
 ('dow', 'int'),
 ('carrier', 'string'),
 ('flight', 'int'),
 ('org', 'string'),
 ('mile', 'int'),
 ('depart', 'double'),
 ('duration', 'int'),
 ('delay', 'int')]

### Loading SMS spam data
The file 'sms.csv' contains a selection of SMS messages which have been classified as either 'spam' or 'ham'. These data have been adapted from the UCI Machine Learning Repository. There are a total of 5574 SMS, of which 747 have been labelled as spam.

Notes on CSV format:

    no header record and
    fields are separated by a semicolon (this is not the default separator).

Data dictionary:

    id — record identifier
    text — content of SMS message
    label — spam or ham (integer; 0 = ham and 1 = spam)


In [182]:
#pnndas is my first love
file_path = './data/sms.csv'
sms_df = pd.read_csv(file_path,sep=';',header=None)
sms_df.head()

,0,1,2
0,1,"Sorry, I'll call later in meeting",0
1,2,Dont worry. I guess he's busy.,0
2,3,Call FREEPHONE 0800 542 0578 now!,1
3,4,Win a 1000 cash prize or a prize worth 5000,1
4,5,"Go until jurong point, crazy.. Available only ...",0


In [183]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Specify column names and types
schema = StructType([
    StructField("id", IntegerType()),
    StructField("text", StringType()),
    StructField("label", IntegerType())
])

file_path = './data/sms.csv'

# Load data from a delimited file
sms = spark.read.csv(file_path, sep=';', header=False, schema=schema)

# Print schema of DataFrame
sms.printSchema()

root
 |-- id: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- label: integer (nullable = true)



### Removing columns and rows

Develop a model which will predict whether or not a given flight will be delayed.  Here we trim those data down by:

    removing an uninformative column
    removing rows which do not have information about whether or not a flight was delayed.


In [184]:
# Remove the 'flight' column
flights = flights.drop('flight')

# Number of records with missing 'delay' values
flights.filter('delay IS NULL').count()

# Remove records with missing 'delay' values
flights = flights.filter('delay IS NOT NULL')

# Remove records with missing values in any column and get the number of remaining rows
flights = flights.dropna()
print(flights.count())

47022


In [185]:
flights.columns

['mon', 'dom', 'dow', 'carrier', 'org', 'mile', 'depart', 'duration', 'delay']

### Column manipulation

The Federal Aviation Administration (FAA) considers a flight to be "delayed" when it arrives 15 minutes or more after its scheduled time.

The next step of preparing the flight data has two parts:

    convert the units of distance, replacing the mile column with a kmcolumn
    create a Boolean column indicating whether or not a flight was delayed.


In [186]:
# Import the required function
from pyspark.sql.functions import round

# Convert 'mile' to 'km' and drop 'mile' column
flights_km = flights.withColumn('km', round(flights.mile * 1.60934, 0)) \
                    .drop('mile')

# Create 'label' column indicating whether flight delayed (1) or not (0)
flights_km = flights_km.withColumn('label', (flights_km.delay >= 15).cast('integer'))

# Check first five records
flights_km.show(5)

+---+---+---+-------+---+------+--------+-----+------+-----+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|
+---+---+---+-------+---+------+--------+-----+------+-----+
|  0| 22|  2|     UA|ORD| 16.33|      82|   30| 509.0|    1|
|  2| 20|  4|     UA|SFO|  6.17|      82|   -8| 542.0|    0|
|  9| 13|  1|     AA|ORD| 10.33|     195|   -5|1989.0|    0|
|  5|  2|  1|     UA|SFO|  7.98|     102|    2| 885.0|    0|
|  7|  2|  6|     AA|ORD| 10.83|     135|   54|1180.0|    1|
+---+---+---+-------+---+------+--------+-----+------+-----+
only showing top 5 rows



### Categorical columns

In the flights data there are two columns, carrier and org, which hold categorical data. Transform those columns into indexed numerical values.

In [187]:
from pyspark.ml.feature import StringIndexer

# Create an indexer
indexer = StringIndexer(inputCol='carrier', outputCol='carrier_idx')

# Indexer identifies categories in the data
indexer_model = indexer.fit(flights_km)

# Indexer creates a new column with numeric index values
flights_km_indexed = indexer_model.transform(flights_km)

# Repeat the process for the other categorical feature
flights_km_indexed = StringIndexer(inputCol='org', outputCol='org_idx').fit(flights_km_indexed).transform(flights_km_indexed)

flights_km_indexed.show(10)

+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|carrier_idx|org_idx|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+
|  0| 22|  2|     UA|ORD| 16.33|      82|   30| 509.0|    1|        0.0|    0.0|
|  2| 20|  4|     UA|SFO|  6.17|      82|   -8| 542.0|    0|        0.0|    1.0|
|  9| 13|  1|     AA|ORD| 10.33|     195|   -5|1989.0|    0|        1.0|    0.0|
|  5|  2|  1|     UA|SFO|  7.98|     102|    2| 885.0|    0|        0.0|    1.0|
|  7|  2|  6|     AA|ORD| 10.83|     135|   54|1180.0|    1|        1.0|    0.0|
|  1| 16|  6|     UA|ORD|   8.0|     232|   -7|2317.0|    0|        0.0|    0.0|
|  1| 22|  5|     UA|SJC|  7.98|     250|  -13|2943.0|    0|        0.0|    5.0|
| 11|  8|  1|     OO|SFO|  7.77|      60|   88| 254.0|    1|        2.0|    1.0|
|  4| 26|  1|     AA|SFO| 13.25|     210|  -10|2356.0|    0|        1.0|    1.0|
|  4| 25|  0|     AA|ORD| 13

## Assembling columns

The final stage of data preparation is to consolidate all of the predictor columns into a single column.

At present our data has the following predictor columns:

    mon, dom and dow
    carrier_idx (derived from carrier)
    org_idx (derived from org)
    km
    depart
    duration


In [188]:
# Import the necessary class
from pyspark.ml.feature import VectorAssembler

# Create an assembler object
assembler = VectorAssembler(inputCols=[
    'mon', 'dom', 'dow', 'carrier_idx', 'org_idx', 'km', 'depart', 'duration'
], outputCol='features')

# Consolidate predictor columns
flights_assembled = assembler.transform(flights_km_indexed)

# Check the resulting column
flights_assembled.select('features', 'delay').show(5, truncate=False)

+-----------------------------------------+-----+
|features                                 |delay|
+-----------------------------------------+-----+
|[0.0,22.0,2.0,0.0,0.0,509.0,16.33,82.0]  |30   |
|[2.0,20.0,4.0,0.0,1.0,542.0,6.17,82.0]   |-8   |
|[9.0,13.0,1.0,1.0,0.0,1989.0,10.33,195.0]|-5   |
|[5.0,2.0,1.0,0.0,1.0,885.0,7.98,102.0]   |2    |
|[7.0,2.0,6.0,1.0,0.0,1180.0,10.83,135.0] |54   |
+-----------------------------------------+-----+
only showing top 5 rows



In [189]:
flights_km_indexed.count()

47022

## Train/test split

In [190]:
flights = flights_assembled

# Split into training and testing sets in a 80:20 ratio
flights_train, flights_test = flights.randomSplit([0.8,0.2],seed=17)

# Check that training set has around 80% of records
training_ratio = flights_train.count() / flights_test.count()
print(training_ratio)

3.9522906793048973


### Build a Decision Tree

In [191]:
# Import the Decision Tree Classifier class
from pyspark.ml.classification import DecisionTreeClassifier

# Create a classifier object and fit to the training data
tree = DecisionTreeClassifier()
tree_model = tree.fit(flights_train)

# Create predictions for the testing data and take a look at the predictions
prediction = tree_model.transform(flights_test)
prediction.select('label', 'prediction', 'probability').show(5, False)

+-----+----------+----------------------------------------+
|label|prediction|probability                             |
+-----+----------+----------------------------------------+
|1    |0.0       |[0.5360414177618479,0.4639585822381521] |
|1    |1.0       |[0.35137538911123817,0.6486246108887618]|
|1    |1.0       |[0.35137538911123817,0.6486246108887618]|
|1    |1.0       |[0.35137538911123817,0.6486246108887618]|
|1    |1.0       |[0.35137538911123817,0.6486246108887618]|
+-----+----------+----------------------------------------+
only showing top 5 rows



### Evaluate the Decision Tree
A confusion matrix gives a useful breakdown of predictions versus known values. It has four cells which represent the counts of:

    True Negatives (TN) — model predicts negative outcome & known outcome is negative
    True Positives (TP) — model predicts positive outcome & known outcome is positive
    False Negatives (FN) — model predicts negative outcome but known outcome is positive
    False Positives (FP) — model predicts positive outcome but known outcome is negative.


In [192]:
# Create a confusion matrix
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TN + TP) / (TN + TP + FN + FP)
print(accuracy)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 1241|
|    0|       0.0| 2485|
|    1|       1.0| 3583|
|    0|       1.0| 2186|
+-----+----------+-----+

0.6390731964191679


### Build a Logistic Regression model

Create a Logistic Regression model on the same data.

Predict whether a flight is likely to be delayed by at least 15 minutes (label 1) or not (label 0).

Use the mon, depart and duration columns for the moment. These are numerical features which can immediately be used for a Logistic Regression model.

In [193]:
# Import the logistic regression class
from pyspark.ml.classification import LogisticRegression

# Create a classifier object and train on training data
logistic = LogisticRegression().fit(flights_train)

# Create predictions for the testing data and show confusion matrix
prediction = logistic.transform(flights_test)
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 1652|
|    0|       0.0| 2645|
|    1|       1.0| 3172|
|    0|       1.0| 2026|
+-----+----------+-----+



### Evaluate the Logistic Regression model

Accuracy is generally not a very reliable metric because it can be biased by the most common target class.

There are two other useful metrics:

    precision and
    recall.

Precision is the proportion of positive predictions which are correct. For all flights which are predicted to be delayed, what proportion is actually delayed?

Recall is the proportion of positives outcomes which are correctly predicted. For all delayed flights, what proportion is correctly predicted by the model?

The precision and recall are generally formulated in terms of the positive target class. But it's also possible to calculate weighted versions of these metrics which look at both target classes.

In [194]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# Calculate precision and recall
precision = TP /(TP + FP)
recall = TP / (TP + FN)
print('precision = {:.2f}\nrecall    = {:.2f}'.format(precision, recall))

# Find weighted precision
multi_evaluator = MulticlassClassificationEvaluator()
weighted_precision = multi_evaluator.evaluate(prediction, {multi_evaluator.metricName: "weightedPrecision"})

# Find AUC
binary_evaluator = BinaryClassificationEvaluator()
auc = binary_evaluator.evaluate(prediction, {binary_evaluator.metricName: "areaUnderROC"})

precision = 0.61
recall    = 0.66


In [195]:
auc

0.6501216177018289

In [196]:
weighted_precision

0.6128474273772628

### Turning Text into Tables
#### Punctuation, numbers and tokens

Use dataset of SMS messages which had been labeled as either "spam" (label 1) or "ham" (label 0), to build a classifier model.

Prepare the SMS messages as follows:

    remove punctuation and numbers
    tokenize (split into individual words)
    remove stop words
    apply the hashing trick
    convert to TF-IDF representation.


In [197]:
sms_df.head()

,0,1,2
0,1,"Sorry, I'll call later in meeting",0
1,2,Dont worry. I guess he's busy.,0
2,3,Call FREEPHONE 0800 542 0578 now!,1
3,4,Win a 1000 cash prize or a prize worth 5000,1
4,5,"Go until jurong point, crazy.. Available only ...",0


In [198]:
file_path = './data/sms2.csv'

# Load data from a delimited file
sms = spark.read.csv(file_path, sep=';',header=True, schema=schema)

sms.show(10)
sms.columns

+---+--------------------+-----+
| id|                text|label|
+---+--------------------+-----+
|  1|Sorry, I'll call ...|    0|
|  2|Dont worry. I gue...|    0|
|  3|Call FREEPHONE 08...|    1|
|  4|Win a 1000 cash p...|    1|
|  5|Go until jurong p...|    0|
|  6|Ok lar... Joking ...|    0|
|  7|Free entry in 2 a...|    1|
|  8|U dun say so earl...|    0|
|  9|Nah I don't think...|    0|
| 10|FreeMsg Hey there...|    1|
+---+--------------------+-----+
only showing top 10 rows



['id', 'text', 'label']

In [199]:
# Import the necessary functions
from pyspark.sql.functions import regexp_replace
from pyspark.ml.feature import Tokenizer

# Remove punctuation (REGEX provided) and numbers
wrangled = sms.withColumn('text', regexp_replace(sms.text, '[_():;,.!?\\-]', ' '))
wrangled = wrangled.withColumn('text', regexp_replace(wrangled.text, '[0-9]', ' '))

# Merge multiple spaces
wrangled = wrangled.withColumn('text', regexp_replace(wrangled.text, ' +', ' '))

# Split the text into words
wrangled = Tokenizer(inputCol='text', outputCol='words').transform(wrangled)

wrangled.show(4, truncate=False)

+---+----------------------------------+-----+------------------------------------------+
|id |text                              |label|words                                     |
+---+----------------------------------+-----+------------------------------------------+
|1  |Sorry I'll call later in meeting  |0    |[sorry, i'll, call, later, in, meeting]   |
|2  |Dont worry I guess he's busy      |0    |[dont, worry, i, guess, he's, busy]       |
|3  |Call FREEPHONE now                |1    |[call, freephone, now]                    |
|4  |Win a cash prize or a prize worth |1    |[win, a, cash, prize, or, a, prize, worth]|
+---+----------------------------------+-----+------------------------------------------+
only showing top 4 rows



### Stop words and hashing

Remove stop words and then apply the hashing trick, converting the results into a TF-IDF.

A quick reminder about these concepts:

      The hashing trick provides a fast and space-efficient way to map a very large (possibly infinite) set of items (in this case, all words contained in the SMS messages) onto a smaller, finite number of values.
    The TF-IDF matrix reflects how important a word is to each document. It takes into account both the frequency of the word within each document but also the frequency of the word across all of the documents in the collection.


In [201]:
sms = wrangled

from pyspark.ml.feature import StopWordsRemover, HashingTF, IDF

# Remove stop words.
wrangled = StopWordsRemover(inputCol='words', outputCol='terms').transform(sms)

# Apply the hashing trick
wrangled = HashingTF(inputCol='terms', outputCol='hash', numFeatures=1024).transform(wrangled)

# Convert hashed symbols to TF-IDF
tf_idf = IDF(inputCol='hash', outputCol='features')\
      .fit(wrangled).transform(wrangled)
      
tf_idf.select('terms', 'features').show(4, truncate=False)

+--------------------------------+----------------------------------------------------------------------------------------------------+
|terms                           |features                                                                                            |
+--------------------------------+----------------------------------------------------------------------------------------------------+
|[sorry, call, later, meeting]   |(1024,[138,344,378,1006],[2.2391682769656747,2.892706319430574,3.684405173719015,4.244020961654438])|
|[dont, worry, guess, busy]      |(1024,[53,233,329,858],[4.618714411095849,3.557143394108088,4.618714411095849,4.937168142214383])   |
|[call, freephone]               |(1024,[138,396],[2.2391682769656747,3.3843005812686773])                                            |
|[win, cash, prize, prize, worth]|(1024,[31,69,387,428],[3.7897656893768414,7.284881949239966,4.4671645129686475,3.898659777615979])  |
+--------------------------------+--------------

In [203]:
tf_idf.select('terms', 'features').show(10, truncate=True)

+--------------------+--------------------+
|               terms|            features|
+--------------------+--------------------+
|[sorry, call, lat...|(1024,[138,344,37...|
|[dont, worry, gue...|(1024,[53,233,329...|
|   [call, freephone]|(1024,[138,396],[...|
|[win, cash, prize...|(1024,[31,69,387,...|
|[go, jurong, poin...|(1024,[116,262,33...|
|[ok, lar, joking,...|(1024,[449,572,66...|
|[free, entry, wkl...|(1024,[16,24,77,1...|
|[u, dun, say, ear...|(1024,[26,212,249...|
|[nah, think, goes...|(1024,[364,396,50...|
|[freemsg, hey, da...|(1024,[112,163,17...|
+--------------------+--------------------+
only showing top 10 rows



### Training a spam classifier

In [211]:
sms = tf_idf

# Split the data into training and testing sets
sms_train, sms_test = sms.randomSplit([0.8, 0.2],seed=13)

# Fit a Logistic Regression model to the training data
logistic = LogisticRegression(regParam=0.2).fit(sms_train)

# Make predictions on the testing data
prediction = logistic.transform(sms_test)

# Create a confusion matrix, comparing predictions to known labels
prediction.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|   47|
|    0|       0.0|  987|
|    1|       1.0|  124|
|    0|       1.0|    3|
+-----+----------+-----+

